In [1]:
import base64
from __future__ import print_function # py 2.7 compat.
import ipywidgets as widgets # Widget definitions.
from traitlets import Unicode # Traitlet needed to add synced attributes to the widget.

This is a custom widget that allows the user to upload file data to the notebook server.  The file data is sent via a statefull `value` attribute of the widget.  The widget has an upload failed event that fires in the front-end and is echoed to the back-end using a custom msg.

In [5]:
import ipywidgets

In [2]:
class FileWidget(widgets.DOMWidget):
    _view_name = Unicode('FilePickerView', sync=True)
    value = Unicode(sync=True)
    filename = Unicode(sync=True)
    
    def __init__(self, **kwargs):
        """Constructor"""
        widgets.DOMWidget.__init__(self, **kwargs) # Call the base.
        
        # Allow the user to register error callbacks with the following signatures:
        #    callback()
        #    callback(sender)
        self.errors = widgets.CallbackDispatcher(accepted_nargs=[0, 1])
        
        # Listen for custom msgs
        self.on_msg(self._handle_custom_msg)

    def _handle_custom_msg(self, content):
        """Handle a msg from the front-end.

        Parameters
        ----------
        content: dict
            Content of the msg."""
        if 'event' in content and content['event'] == 'error':
            self.errors()
            self.errors(self)

            
            
            

In [3]:
%%javascript

require(["widgets/js/widget", "widgets/js/manager"], function(widget, manager){

    var FilePickerView = widget.DOMWidgetView.extend({
        render: function(){
            // Render the view.
            this.setElement($('<input />')
                .attr('type', 'file'));
        },
        
        events: {
            // List of events and their handlers.
            'change': 'handle_file_change',
        },
       
        handle_file_change: function(evt) { 
            // Handle when the user has changed the file.
            
            // Retrieve the first (and only!) File from the FileList object
            var file = evt.target.files[0];
            if (file) {

                // Read the file's textual content and set value to those contents.
                var that = this;
                var file_reader = new FileReader();
                file_reader.onload = function(e) {
                    that.model.set('value', e.target.result);
                    that.touch();
                }
                file_reader.readAsText(file);
            } else {

                // The file couldn't be opened.  Send an error msg to the
                // back-end.
                this.send({ 'event': 'error' });
            }

            // Set the filename of the file.
            this.model.set('filename', file.name);
            this.touch();
        },
    });
        
    // Register the FilePickerView with the widget manager.
    manager.WidgetManager.register_widget_view('FilePickerView', FilePickerView);
});

<IPython.core.display.Javascript object>

The following shows how the file widget can be used.

In [4]:
file_widget = FileWidget()

# Register an event to echo the filename when it has been changed.
def file_loading():
    print("Loading %s" % file_widget.filename)
file_widget.on_trait_change(file_loading, 'filename')

# Register an event to echo the filename and contents when a file
# has been uploaded.
def file_loaded():
    print("Loaded, file contents: %s" % file_widget.value)
file_widget.on_trait_change(file_loaded, 'value')

# Register an event to print an error message when a file could not
# be opened.  Since the error messages are not handled through
# traitlets but instead handled through custom msgs, the registration
# of the handler is different than the two examples above.  Instead
# the API provided by the CallbackDispatcher must be used.
def file_failed():
    print("Could not load file contents of %s" % file_widget.filename)
file_widget.errors.register_callback(file_failed)

file_widget

Loading screencap_chaste.png
Loaded, file contents: �PNG

IHDR  q  8   �ײ   	pHYs     ��   tIME�
'��.�    IDATx���vI��������dDMfwOל9����綻޾����b%H������\���@��`���t�bnE���X�����͛�p��c! ����O~V�?��9�?>���q�q�������������x��'�q�q�q�q�r�q�q�q�r�q�q�q�r�q�q����~
����˲"����� }Dp�����x�G/���8��8�hx��q�q�q���:��8����0�e�� ����q�q����T��8��8��8�T��8��8��8|��8���)�>t�a����,�8��8��N�8�7�ڔ���8��8�T��|�N�г7odrDgɭ���u;r*@Dt�t��/r(�d��3����q�qw*�q�����M�I���8�"��	VD��@/I9��8��N�8�7ç	��\�`0S����L�ةVm�F����<E��Q���h�S^�K�KX��8��S9��`E��E��Tup*�c�6��`U�4
�bX*m�G��̠
�z�4s����;��8��S9���Tw�co:T�F)6���m}�Re���X�����+��� ���)�s��<����6/3�tv���TaS�ߏ-^� ��31&f
�YR�GY�}6t�qǝ�q�o[��n>��N5l���\��N�j웚��{>Lv�Q���;�C0�f
ʖ���M���`КE^�g�}7e�:��^�TG{�Fs�V�z����7����THb}.7�2���WP}Tc��S�f
2-�K^E1����8��!Ê=3�q�qܩ�����.��ڟ~X;�;4$���"�ũj���I4�;�U��?�ѡ�S���WeT5��M|6�O�m�f�AV���0�y\�ѳL�6v�J���1U��1�@�9